In [6]:
import pandas as pd
import numpy as np
import sys
import os
import scipy.stats as stats
from statistics import mean
from enum import Enum
import sys
print(sys.version)

class DB(Enum):
    Mcyt=0
    eBioSignDS1=1
    eBioSignDS2=3
    BiosecurID=4
    BiosecureDS2=5
    EvalDB=6

class Signature:
    def __init__(self, data, filePath):
        self.data = data
        self.filePath = filePath
        pathParts= filePath.split('/')
        self.inputDevice=pathParts[2]
        if "signature" in filePath:
            self.db='EvalDb'
            self.inputDevice='Unknown'
            return
        
        parts=pathParts[4].split('_')
        signerId=parts[0][1:]
        self.db=self.getDatabase(pathParts[2],pathParts[3],int(signerId))
        print(self.db)
        pressureColumn=self.getPressureColumn()
        
        self.x=data[:, 0]
        self.y=data[:, 1]
        self.t=data[:, 2]
        self.pressure=data[:, pressureColumn]
        
        if (self.inputDevice == "Unkown"):
            seelf.inputDevice=getInputDeviceForEvalDb()
        
        self.correctZeroTime()
        self.correctSameTimestamps()
    
        if self.inputDevice == 'stylus':
            i=0
            while len(self.pressure)>0 and self.pressure[0] == 0:
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)

            while (lines.Count > 0 and self.pressure[len(self.pressure)-1].Pressure == 0):
                self.x.pop(0)
                self.y.pop(0)
                self.t.pop(0)
                self.pressure.pop(0)
    
    def getPressureColumn(self):
        if self.db==DB.Mcyt:
            return 5
        elif self.db==DB.BiosecurID:
            return 6
        elif self.db==DB.BiosecureDS2:
            return 6
        elif self.db==DB.eBioSignDS1:
            return 3
        elif self.db==DB.eBioSignDS2:
            return 3
        elif self.db==DB.EvalDB:
            return 3
        
    def getDatabase(self, split, inputDevice, signerId):
        print(split)
        print(inputDevice)
        print(signerId)
        if split=='Development':
            if inputDevice=='finger':
                if 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=230: 
                    return DB.Mcyt
                elif 231<=signerId<=498:
                    return DB.BiosecurID
                elif 1009<=signerId<=1038:
                    return DB.eBioSignDS1
                elif 1039<=signerId<=1084:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        elif split=='Evaluation':
            if inputDevice=='finger':
                if 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            elif inputDevice=="stylus":
                if 1<=signerId<=100:
                    return DB.Mcyt
                elif 101<=signerId<=232:
                    return DB.BiosecurID
                elif 233<=signerId<=372:
                    return DB.BiosecureDS2
                elif 373<=signerId<=407:
                    return DB.eBioSignDS1
                elif 408<=signerId<=442:
                    return DB.eBioSignDS2
                else:
                    print("Undefined DB for file: {file}")
            else:
                print("Undefined InputDevice for file: {file}")
        else:
            print("Undefined InputDevice for file: {file}")
            
    def getInputDeviceForEvalDb(self):
        for i in range(0, len(self.pressure), 1):
            if self.pressure[i]!=0:
                return False

        return True
    
    def isAllZero(self):
        for i in range(0, self.data.shape[0], 1):
            if self.data[i,0]!=0:
                return False

        return True
    
    def correctZeroTime(self):
        if self.isAllZero():
            for i in range(0, self.data.shape[0], 1):
                self.data[i,0]=i*10
                
    def correctSameTimestamps(self):
        i=0
        while i < self.data.shape[0]-1:
            if self.data[i,0]==self.data[i+1,0]:
                self.data=np.delete(self.data, i, 0)
            else:
                i += 1

def readFilesInDir(dirPath, allFiles):
    signatureFiles = os.listdir(dirPath)
    for file in signatureFiles:
        dataframe=pd.read_csv(dirPath + '/' + file,sep=' ',skiprows=[0],header=None)
        array=np.array(dataframe)
        signature=Signature(array,dirPath + '/' + file)
        allFiles.append(signature)

def Average(lst):
    return sum(lst) / len(lst)

def normalize(allFiles):
    xAll=[]
    for item in allFiles:
        xAll+item.x
        
    print('xAll')
    print(xAll)
    average=mean(xAll)
    z = stats.zscore(XAll)
    print(average)
    print(z)
allFiles=[]
        
#readFilesInDir('C:\DeepSignDB/Evaluation/stylus',allFiles)
#readFilesInDir('C:\DeepSignDB/Evaluation/finger',allFiles)
#readFilesInDir('C:\DeepSignDB/Development/stylus',allFiles)

readFilesInDir('D:/DeepSignSmall/Evaluation/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Evaluation/finger',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/stylus',allFiles)
readFilesInDir('D:/DeepSignSmall/Development/finger',allFiles)
            
for file in allFiles:
    #print(file.data)
    print(file.filePath)
    print(file.db)
    print(file.pressure)
    
print('Allfiles:')
print(allFiles)
print('Loading data finished')

normalize(allFiles)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Evaluation
stylus
1
DB.Mcyt
Evaluation
stylus
1
DB.Mcyt
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
275
DB.BiosecureDS2
Evaluation
stylus
356
DB.BiosecureDS2
Evaluation
stylus
356
DB.BiosecureDS2
Evaluation
stylus
408
DB.eBioSignDS2
Evaluation
stylus
408
DB.eBioSignDS2
Evaluation
stylus
442
DB.eBioSignDS2
Evaluation
finger
373
DB.eBioSignDS1
Evaluation
finger
373
DB.eBioSignDS1
Evaluation
finger
373
DB.eBioSignDS1
Evaluation
finger
387
DB.eBioSignDS1
Evaluation
finger
387
DB.eBioSignDS1
Evaluation
finger
428
DB.eBioSignDS2
Evaluation
finger
428
DB.eBioSignDS2
Development
stylus
1
DB.Mcyt
Development
stylus
139
DB.Mcyt
Development
stylus
289
DB.BiosecurID
Development
stylus
327
DB.Bios

ValueError: operands could not be broadcast together with shapes (0,) (96,) 